In [1]:
import pandas as pd
import numpy as np
from gensim.parsing.preprocessing import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

df_corpus = pd.read_csv('corpus.csv',index_col=0)

corpus = []
for topic in df_corpus:
    corpus += df_corpus[topic].tolist()

answ_corpus = pd.Series(corpus)
answ_corpus = answ_corpus.apply(strip_tags)
answ_corpus = answ_corpus.apply(strip_multiple_whitespaces)
corpus = pd.Series(corpus)
corpus = corpus.apply(lambda x: x.lower())
corpus = corpus.apply(strip_tags)
corpus = corpus.apply(strip_punctuation2)
corpus = corpus.apply(strip_numeric)
corpus = corpus.apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u' ', x))
corpus = corpus.apply(lambda x: re.sub('«|»', ' ', x))
corpus = corpus.apply(strip_multiple_whitespaces)



import stop_words
from nltk.stem.snowball import RussianStemmer

corpus_tokens = []
inds_to_drop = []

for i, sentence in enumerate(corpus[:]):
    tmp_tokens = []
    sp = sentence.split()
    for word in sp:
        if word not in stop_words.get_stop_words('ru'):
            if morph.word_is_known(word):
                tmp_tokens.append(word)
    if len(tmp_tokens) > 0:
        corpus_tokens.append(tmp_tokens)
    else:
        inds_to_drop.append(i)
        
print(len(corpus_tokens))


stemmer = RussianStemmer()

corpus_tokens_stem = []

for i, tokens in enumerate((corpus_tokens[:])):
    tmp = [stemmer.stem(word) for word in tokens]
    corpus_tokens_stem.append(tmp)
    
print(len(corpus_tokens_stem))


clear_corpus = []
for token_list in corpus_tokens_stem:
    if len(' '.join(token_list))<2:
        print(token_list)
    clear_corpus.append(' '.join(token_list))
clear_corpus[0]


vectorizer = TfidfVectorizer()
vectorizer.fit(clear_corpus)
X = vectorizer.transform(clear_corpus)


800
800


In [2]:
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
N_TOPICS = 4
max_iteration = 100
batch_size = 15
evaluate_every=1
verb=1
learning_method='batch'

In [4]:
lda = LatentDirichletAllocation(n_components=N_TOPICS, max_iter=max_iteration, batch_size=batch_size, evaluate_every=evaluate_every, verbose=verb, 
                                        n_jobs=4, learning_method=learning_method)

In [5]:
lda.fit(X)

iteration: 1 of max_iter: 100, perplexity: 27600.9063
iteration: 2 of max_iter: 100, perplexity: 25580.9740
iteration: 3 of max_iter: 100, perplexity: 23735.5510
iteration: 4 of max_iter: 100, perplexity: 21765.0470
iteration: 5 of max_iter: 100, perplexity: 19935.3627
iteration: 6 of max_iter: 100, perplexity: 18475.8418
iteration: 7 of max_iter: 100, perplexity: 17352.7465
iteration: 8 of max_iter: 100, perplexity: 16622.6509
iteration: 9 of max_iter: 100, perplexity: 16238.9430
iteration: 10 of max_iter: 100, perplexity: 15970.6553
iteration: 11 of max_iter: 100, perplexity: 15741.7401
iteration: 12 of max_iter: 100, perplexity: 15616.6990
iteration: 13 of max_iter: 100, perplexity: 15549.1735
iteration: 14 of max_iter: 100, perplexity: 15530.5198
iteration: 15 of max_iter: 100, perplexity: 15507.7614
iteration: 16 of max_iter: 100, perplexity: 15490.3553
iteration: 17 of max_iter: 100, perplexity: 15476.5739
iteration: 18 of max_iter: 100, perplexity: 15448.1999
iteration: 19 of ma

LatentDirichletAllocation(batch_size=15, doc_topic_prior=None,
             evaluate_every=1, learning_decay=0.7, learning_method='batch',
             learning_offset=10.0, max_doc_update_iter=100, max_iter=100,
             mean_change_tol=0.001, n_components=4, n_jobs=4,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=1)

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
svd = TruncatedSVD(n_components=N_TOPICS, n_iter=100).fit(X)

lsa = make_pipeline(svd, Normalizer(copy=False))

In [7]:
keys = list(vectorizer.vocabulary_.keys())
values = list(vectorizer.vocabulary_.values())

_ind = np.argsort(values)
words_sorted = np.asarray(keys)[_ind]

print(len(keys), len(values), len(words_sorted))

8757 8757 8757


In [8]:
df_corpus.columns

Index(['14', '3', '5', '7'], dtype='object')

In [9]:
import numpy as np
stub = [0] * N_TOPICS
threshold = 0.1

for i in range(N_TOPICS):
    
    szd = pd.Series(np.sort(svd.components_[i])[-10:])
    szd_mask = szd.apply(lambda x: False if x<threshold else True)
    
    print(szd[szd_mask].values)
    
    ind = np.argsort(lda.components_[i])[-szd[szd_mask].shape[0]:]
    print(words_sorted[ind])
    
    print()
#     break

[0.10326071 0.10963394 0.11180692 0.11761075 0.11883023 0.13811186
 0.1778697  0.18489953 0.19264438]
['прощан' 'войн' 'стар' 'артист' 'приключен' 'вахтангов' 'актер' 'театр'
 'этуш']

[0.10341654 0.10780936 0.12882586 0.20553013 0.20749934]
['кровел' 'ямочн' 'вывезл' 'автовышк' 'куб']

[0.10743302 0.10854844 0.12337026 0.12765806 0.15341745 0.16029703
 0.20743902 0.34525624]
['московск' 'компан' 'пресс' 'росс' 'руб' 'служб' 'тыс' 'москв']

[0.10356885 0.10890642 0.1114912  0.11399445 0.13236054 0.14288891
 0.1660668  0.1968348  0.27397104 0.34109378]
['опроверг' 'беремен' 'неправд' 'шутк' 'тянет' 'нейрохирург' 'враг'
 'распуст' 'спин' 'назар']



In [ ]:
# политика # экономика # технологии # общество 